# DARPA-ARC Notebook 4: fMRI Second Levels

## Precompute Permutations
Based on intial calculations, we assume one full loop of WLS + TFCE will take ~17s. We will submit jobs of 100 iterations (approx. 30 minutes time on cluster).

In [ ]:
import os
import numpy as np
np.random.seed(47404)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

n_subj = 28
n_permutations = 5000
inc = 100

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Generate permutations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = []
while True:
    arr = np.random.choice([1,-1],n_subj,replace=True)
    if not np.any(np.apply_along_axis(np.array_equal, 0, permutations, arr)): 
        permutations.append(arr)
    if len(permutations) >= n_permutations: 
        break 
        
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = np.array(permutations)
index = np.arange(0,n_permutations+1,inc)
for n, ix in enumerate(index[1:]): 
    np.save(os.path.join('fmri_second_levels', 'permutations', 'sign_flips_%s' %(n+1)), permutations[ix-inc:ix])
            
print 'Done.'

## Make Surface Masks

In [ ]:
import os
import numpy as np
import nibabel as nib
from mne import read_label, read_surface, spatial_tris_connectivity, set_log_level
set_log_level(verbose=False) 
subj_dir = '/space/lilli/1/users/DARPA-Recons/fscopy'
mri_dir = '/autofs/space/lilli_002/users/DARPA-ARC/NN_bayes_2016/FINAL6/'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

label_dir = os.path.join(subj_dir, 'label', 'dkt40')

roi_list = ['caudalanteriorcingulate', 'rostralanteriorcingulate', 'posteriorcingulate',
            'superiorfrontal', 'medialorbitofrontal', 'rostralmiddlefrontal', 'caudalmiddlefrontal',
            'parsopercularis', 'parstriangularis', 'parsorbitalis', 'lateralorbitofrontal', 'insula']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Make labels.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 

for hemi in ['lh', 'rh']:
    
    ## Assemble and merge labels.
    label = []
    for roi in roi_list: label.append(read_label(os.path.join(label_dir,'%s-%s.label' %(roi,hemi))))
    label = np.sum(label)
    
    ## Save label.
    label.name = 'arc-%s' %hemi
    label.save('fmri_second_levels/arc-%s.label' %hemi)
    
    ## Load surface.
    _, tris = read_surface(os.path.join(subj_dir, 'surf', '%s.white' %hemi))
    mapping = np.in1d(np.unique(tris),label.vertices)
    
    ## Reduce triangles to those in label.
    ix = np.all(np.apply_along_axis(np.in1d, 0, tris, label.vertices), axis=1)
    tris = tris[ix]

    ## Compute connectivity.
    coo = spatial_tris_connectivity(tris, remap_vertices=True)
    np.savez('fmri_second_levels/%s_connectivity' %hemi, data = coo.data, row = coo.row,
             col = coo.col, shape = coo.shape, mapping=mapping, vertices=label.vertices)
    
print 'Done.'

## Make Volume Mask

In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.sparse import coo_matrix
lut = '/usr/local/freesurfer/stable5_3_0/FreeSurferColorLUT.txt'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define subcortical structures of interest.
roi_dict = {18:'Left-Amygdala', 11:'Left-Caudate', 17:'Left-Hippocampus', 12:'Left-Putamen', 
            54:'Right-Amygdala', 50:'Right-Caudate', 53:'Right-Hippocampus', 51:'Right-Putamen'}

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Create mask.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load aseg.
aseg = '/space/lilli/1/users/DARPA-Recons/fsaverage/mri.2mm/aseg.mgz'
aseg = nib.load(aseg).get_data()

## Find all voxels in ROI list. Get corresponding labels.
mapping = np.in1d(aseg, roi_dict.keys()).reshape(aseg.shape)
voxels = np.where(mapping)
names = np.array([roi_dict[i] for i in aseg[voxels]])
voxels = np.vstack(voxels).T

## Initialize connectivity matrix.
n_voxels, _ = voxels.shape
coo = np.zeros([n_voxels,n_voxels], dtype=int)

## Iteratively test for adjacency.
## Here we use 6-lattice connectivity (up,down,forward,backward,left,right).
for n in range(n_voxels):
    diff = np.linalg.norm(voxels - voxels[n], axis=1)
    M, = np.where(diff==1.)
    for m in M: coo[n,m] = 1 
coo = coo_matrix(coo)
    
## Save.
np.savez('fmri_second_levels/mni305_connectivity', data = coo.data, row = coo.row,
         col = coo.col, shape = coo.shape, mapping=mapping, voxels=voxels, names=names)
print 'Done.'

## Extract Mean Signal from ROIs
Necessary for computing percent signal change down the line.

In [ ]:
import os
import numpy as np
import nibabel as nib
from pandas import read_csv
mri_dir = 'fmri_first_levels/'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
fd = 0.9

n_acq = 977
tr = 1.75

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Get subjects list.
info = read_csv('demographics.csv')
subjects = info.loc[~info.Exlude, 'Subject'].as_matrix()

## Define TR onsets.
tr_onsets = np.insert( np.cumsum( np.ones(n_acq - 1) * tr ), 0, 0 )

mean_signal = dict()
for space in ['lh','rh','mni305']:
    
    print space,
    
    ## Load masks.
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    include = npz['mapping']
    
    ## Preallocate space.
    ms = np.zeros([len(subjects), include.sum()])
    
    ## Iterate over subjects.
    for n, subject in enumerate(subjects):
        
        ## Load data.
        subj_dir = os.path.join(mri_dir, subject, 'arc_001', '001')
        if space == 'mni305': f = os.path.join(subj_dir,'fmcpr.sm%s.%s.2mm.b0dc.nii.gz' %(sm,space))
        else: f = os.path.join(subj_dir,'fmcpr.sm%s.fsaverage.%s.b0dc.nii.gz' %(sm,space))
        data = nib.load(f).get_data()
        
        ## Censor data. Average across acquisitions.
        try: censor = np.loadtxt(os.path.join(subj_dir, 'FINAL.censor.%s.par' %fd))
        except IOError: censor = []
        censor = np.invert(np.in1d(tr_onsets, censor))
        
        data = data[include,...].squeeze()
        data = data[...,censor].mean(axis=1)
        
        ## Append.
        ms[n] = data
    
    ## Store in dictionary.
    mean_signal[space] = ms
    
## Save.
f = 'fmri_second_levels/mean_signal'
np.savez_compressed(f, lh = mean_signal['lh'], rh = mean_signal['rh'], mni305 = mean_signal['mni305'])
print 'Done.'

## Assemble Data

In [ ]:
import os
import numpy as np
import nibabel as nib
from mne import read_label
from pandas import read_csv
mri_dir = 'fmri_first_levels/concat-sess/FINAL2/'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
spaces = ['lh','rh','mni305']
thresholds = [0.9]
contrasts = ['Delib', 'DelibMod', 'Antcp', 'AntcpMod', 'Shock']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for space in spaces:

    ## Load masks.
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    include = npz['mapping']
    
    for fd in thresholds:
                
        results_dir = os.path.join(mri_dir, 'FINAL2.%s.%s.%s' %(sm,fd,space))
            
        for contrast in contrasts:
        
            ## Make save directory.
            out_dir = os.path.join('fmri_second_levels', 'FINAL2.%s.%s.%s.%s' %(sm,fd,space,contrast))
            if not os.path.isdir(out_dir): os.makedirs(out_dir)
    
            ## Load data.
            ces = nib.load(os.path.join(results_dir, 'FINAL2.%s.par' %contrast, 'ces.nii.gz')).get_data()
            cesvar = nib.load(os.path.join(results_dir, 'FINAL2.%s.par' %contrast, 'cesvar.nii.gz')).get_data()
            affine = nib.load(os.path.join(results_dir, 'FINAL2.%s.par' %contrast, 'ces.nii.gz')).affine
            
            ## Masking.
            ces = ces[include,...]
            cesvar = cesvar[include,...]
                    
            ## Save.
            np.savez_compressed(os.path.join(out_dir, 'first_levels'), ces=ces.squeeze(), cesvar=cesvar.squeeze())
            np.save(os.path.join(out_dir, 'affine'), affine)

print 'Done.'

## Perform WLS Permutations

In [ ]:
import os, sys
import numpy as np
from pandas import read_csv
from scipy.sparse import coo_matrix
from mne.stats.cluster_level import _find_clusters as find_clusters
root_dir = '/space/sophia/2/users/DARPA-Behavior/notebooks/bayes/decision_making/NN_bayes_2016'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sm = 6
fd = 0.9
space = 'mni305'
contrast = 'Delib'

## Permutation parameters.
permutations = 0

## TFCE parameters.
threshold = dict(start=0.1, step=0.1, h_power=2, e_power=0.5)
tail = 0
max_step = 1

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

def load_sparse_coo(filename):
    npz = np.load(filename)
    M,N = npz['shape']
    return coo_matrix( (npz['data'], (npz['row'],npz['col'])), (M,N) )

out_dir = os.path.join(root_dir, 'fmri_second_levels', 'FINAL.%s.%s.%s.%s' %(sm,fd,space,contrast))

## Load data.
npz = np.load(os.path.join(out_dir, 'first_levels.npz'))
ces = npz['ces']
cesvar = np.abs( 1. / npz['cesvar'] )

## Define indices.
connectivity = load_sparse_coo(os.path.join(root_dir, 'fmri_second_levels', '%s_connectivity.npz' %space))
index,  = np.where(~np.isinf(cesvar).sum(axis=1).astype(bool))
include = ~np.isinf(cesvar).sum(axis=1).astype(bool)
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup for permutation testing.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load subject information.
info = read_csv(os.path.join(root_dir, 'demographics.csv'))
info = info[~info.Exlude].reset_index()
n_subj, _ = info.shape

## Build Design Matrix.
X = np.zeros((n_subj,2))
X[:,0] = 1                                        # Intercept
X[:,1] = np.where(info.Scanner == 'Trio', 0, 1)   # Scanner
n_subj, n_pred = X.shape

## If specified, load precomputed sign flips.
if permutations: sign_flips = np.load(os.path.join(root_dir, 'fmri_second_levels', 'permutations', 'sign_flips_%s.npy' %permutations))
else: sign_flips = np.ones((1,n_subj))
n_shuffles = sign_flips.shape[0]

## Preallocate arrays for results.
shape = [n_shuffles] + list(ces.shape[:-1])
Bmap = np.zeros(shape)
Fmap = np.zeros(shape)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

'''
Following the instructions of Winkler et al. (2014), we use the Freedman and Lane (1983)
permutation procedure. This allows us to precompute a number of values ahead of time.

To understand the WLS computations, please see:
https://github.com/statsmodels/statsmodels/blob/master/statsmodels/base/model.py
https://github.com/statsmodels/statsmodels/blob/master/statsmodels/regression/linear_model.py
'''

def wls(X,Y,W):
    B = np.linalg.inv(X.T.dot(W).dot(X)).dot(X.T).dot(W).dot(Y)
    ssr = W.dot( np.power(Y - np.dot(X,B),2) ).sum()
    scale = ssr / (n_subj - n_pred)
    cov_p = np.linalg.inv(X.T.dot(W).dot(X)) * scale
    F = np.power(B[0],2) * np.power(cov_p[0,0],-1)
    return B[0], F

## Loop it!
for n, sf in enumerate(sign_flips):
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    for m in index:

        ## Update variables.
        W = np.diag(cesvar[m])
        Y = ces[m]
        
        ## Permute values.
        ## See Winkler et al. (2014), pg. 385
        ## To compute Hat Matrix, see: https://en.wikipedia.org/wiki/Projection_matrix and 
        Z = X[:,1:]
        ZZ = Z.dot( np.linalg.inv( Z.T.dot(W).dot(Z) ) ).dot(Z.T).dot(W)
        Rz = np.identity(n_subj) - ZZ
        Y = np.diag(sf).dot(Rz).dot(Y)
        
        ## Perform WLS.
        Bmap[n,m], Fmap[n,m] = wls(X,Y,W) 

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform TFCE.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    _, Fmap[n] = find_clusters(Fmap[n], threshold, tail=tail, connectivity=connectivity, 
                               include=include, max_step=max_step, show_info=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save results.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#  

if not permutations: f = os.path.join(out_dir, '%s_%s_obs' %(space, contrast))
else: f = os.path.join(out_dir, '%s_%s_perm-%s' %(space, contrast, permutations)) 
np.savez_compressed(f, Bmap=Bmap, Fmap=Fmap)
    
print 'Done.'

## Perform FWE Corrections

In [ ]:
import os
import numpy as np
import nibabel as nib

def prepare_image(arr, space):
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    image = np.zeros_like(npz['mapping'], dtype=float)
    
    if not space == 'mni305': 
        image[npz['vertices']] += arr
    else:
        x,y,z = npz['voxels'].T
        image[x,y,z] += arr
    
    for _ in range(4 - len(image.shape)): image = np.expand_dims(image,-1)
    return image

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
fd = 0.9
spaces = ['lh','rh','mni305']
contrasts = ['Delib','DelibMod','Antcp','AntcpMod','Shock']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = np.arange(50) + 1

for contrast in contrasts:

    print contrast,
    
    for n, space in enumerate(spaces):
    
        out_dir = 'fmri_second_levels/FINAL2.%s.%s.%s.%s' %(sm, fd, space, contrast)
        
        ## Load true effects.
        npz = np.load(os.path.join(out_dir, '%s_%s_obs.npz' %(space,contrast)))
        Bmap = npz['Bmap'].squeeze()
        Fmap = npz['Fmap'].squeeze()
        
        ## Load permutations.
        Pmap = []
        for p in permutations: 
            npz = np.load(os.path.join(out_dir, '%s_%s_perm-%s.npz' %(space,contrast,p)))
            Pmap.append(npz['Fmap'])
        Pmap = np.concatenate(Pmap, axis=0)
        n_permutations, _ = Pmap.shape

        ## Compute p-values via FWE.
        p_values = np.ones_like(Fmap)
        for mp in Pmap.max(axis=1): p_values += mp > Fmap
        p_values /= n_permutations + 1.
        p_values = -np.log10(p_values) * np.sign(Bmap)
      
        ## Save maps.
        np.save(os.path.join(out_dir,'%s_%s_fwe' %(space,contrast)), p_values)
        for arr, name in zip([Bmap,Fmap,p_values],['beta','F','fwe']):
            image = prepare_image(arr, space)
            image = nib.Nifti1Image(image, np.load(os.path.join(out_dir,'affine.npy')))
            nib.save(image, os.path.join(out_dir, '%s.nii.gz' %name))
            
print 'Done.'

## Perform FDR Corrections
Not used, but programmed for example's sake.

In [ ]:
import os
import numpy as np
import nibabel as nib
from mne.stats import fdr_correction

def prepare_image(arr, space):
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    image = np.zeros_like(npz['mapping'], dtype=float)
    
    if not space == 'mni305': 
        image[npz['vertices']] += arr
    else:
        x,y,z = npz['voxels'].T
        image[x,y,z] += arr
    
    for _ in range(4 - len(image.shape)): image = np.expand_dims(image,-1)
    return image

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
fd = 0.9
spaces = ['lh','rh','mni305']
contrasts = ['Delib','DelibMod','Antcp','AntcpMod','Shock']
contrasts = ['DelibMod']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = np.arange(50) + 1

for contrast in contrasts:

    print contrast,
    FDR, signs = [], []
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute p-values within spaces.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    for n, space in enumerate(spaces):
    
        out_dir = 'fmri_second_levels/FINAL.%s.%s.%s.%s' %(sm, fd, space, contrast)
        
        ## Load true effects.
        npz = np.load(os.path.join(out_dir, '%s_%s_obs.npz' %(space,contrast)))
        Bmap = npz['Bmap'].squeeze()
        Fmap = npz['Fmap'].squeeze()
        
        ## Load permutations.
        Pmap = []
        for p in permutations: 
            npz = np.load(os.path.join(out_dir, '%s_%s_perm-%s.npz' %(space,contrast,p)))
            Pmap.append(npz['Fmap'])
        Pmap = np.concatenate(Pmap, axis=0)
        n_permutations, _ = Pmap.shape

        ## Compute p-values via FWE.
        p_values = (Pmap >= Fmap).sum(axis=0) + 1.
        p_values /= n_permutations + 1.
        FDR.append(p_values)
        signs.append(np.sign(Bmap))
    
        ## Save maps.
        for arr, name in zip([Bmap,Fmap],['beta','F']):
            image = prepare_image(arr, space)
            image = nib.Nifti1Image(image, np.load(os.path.join(out_dir,'affine.npy')))
            nib.save(image, os.path.join(out_dir, '%s.nii.gz' %name))        

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform FDR corrections.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            
    ## Assemble info.
    indices = np.concatenate([np.ones_like(arr) * n for n, arr in enumerate(FDR)])
    FDR = np.concatenate(FDR)
    signs = np.concatenate(signs)

    ## Perform FDR correction.
    FDR[np.where(signs)] = fdr_correction(FDR[np.where(signs)])[-1]
    FDR = -np.log10(FDR) * signs

    ## Save maps.
    for n, space in enumerate(spaces):
        out_dir = 'fmri_second_levels/FINAL.%s.%s.%s.%s' %(sm, fd, space, contrast)
        np.save(os.path.join(out_dir,'%s_%s_fdr' %(space,contrast)), FDR[indices==n])
        image = prepare_image(FDR[indices==n], space)
        image = nib.Nifti1Image(image, np.load(os.path.join(out_dir,'affine.npy')))
        nib.save(image, os.path.join(out_dir, 'fdr.nii.gz'))
    
print 'Done.'

# Section 5: Visualization

## Threshold Second-Level Maps
Thresholding clusters such that:
* p < 0.05 (FWE corrected, alpha = 0.05)
* Surface: clusters > 100mm2
* Volume: clusters > 20 contiguous voxels

In [ ]:
import os
import numpy as np
import nibabel as nib
from pandas import DataFrame
from scipy.sparse import coo_matrix
from mne.stats.cluster_level import _find_clusters as find_clusters
fs_dir = 'recons'

def load_sparse_coo(filename):
    npz = np.load(filename)
    M,N = npz['shape']
    return coo_matrix( (npz['data'], (npz['row'],npz['col'])), (M,N) )

def prepare_image(arr, space):
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    image = np.zeros_like(npz['mapping'], dtype=float)
    
    if not space == 'mni305': 
        image[npz['vertices']] += arr
    else:
        x,y,z = npz['voxels'].T
        image[x,y,z] += arr
    
    for _ in range(4 - len(image.shape)): image = np.expand_dims(image,-1)
    return image

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sm = 6
fd = 0.9
spaces = ['lh','rh','mni305']
contrasts = ['Delib','DelibMod','Antcp','AntcpMod','Shock']

## Thresholding parameters.
threshold = -np.log10( 0.05 )
min_cluster = dict(lh = 100, rh = 100, mni305 = 20)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for space in spaces:
    
    print space,
    
    ## Load connectivity information.
    connectivity = load_sparse_coo('fmri_second_levels/%s_connectivity.npz' %space)

    ## Load mapping information.
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    
    if not space == 'mni305':
        vertices = npz['vertices']
        average_area = nib.load(os.path.join(fs_dir, 'fsaverage', 'surf', '%s.white.avg.area.mgh' %space)).get_data()
        average_area = average_area[vertices].squeeze()
        
    for contrast in contrasts:
        
        ## Load FWE-corrected p-values.
        f = 'fmri_second_levels/FINAL2.%s.%s.%s.%s/%s_%s_fwe.npy' %(sm, fd, space, contrast, space, contrast)
        fwe = np.load(f)
        
        ## Find clusters.
        include = np.where(fwe,True,False)
        clusters, sums = find_clusters(fwe, threshold, tail=0, connectivity=connectivity, include=include, t_power=0)
        
        ## Compute areas.
        if not space == 'mni305': cluster_sums = np.array([average_area[c].sum() for c in clusters])
        else: cluster_sums = sums
            
        ## Threshold.
        try:
            survival_ix = np.concatenate([c for c, s in zip(clusters,cluster_sums) if s > min_cluster[space]])
            fwe[~np.in1d(np.arange(fwe.shape[0]), survival_ix)] = 0
        except ValueError:
            fwe = np.zeros_like(fwe)
        
        ## Save.
        image = prepare_image(fwe, space)
        image = nib.Nifti1Image(image, np.load(os.path.join(os.path.dirname(f),'affine.npy')))
        nib.save(image, os.path.join(os.path.dirname(f), 'fwe_thresh_%0.3f.nii.gz' %threshold))
        
print 'Done.'

## Compute Percent Signal Change

In [ ]:
import os
import numpy as np
import nibabel as nib
from pandas import read_csv
mri_dir = 'fmri_first_levels'

def prepare_image(arr, space):
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    image = np.zeros_like(npz['mapping'], dtype=float)
    
    if not space == 'mni305': 
        image[npz['vertices']] += arr
    else:
        x,y,z = npz['voxels'].T
        image[x,y,z] += arr
    
    for _ in range(4 - len(image.shape)): image = np.expand_dims(image,-1)
    return image

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sm = 6
fd = 0.9
spaces = ['lh','rh','mni305']
contrasts = ['Delib','DelibMod','Antcp','AntcpMod','Shock']
threshold = 1.301

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Get subjects list.
info = read_csv('demographics.csv')
subjects = info.loc[~info.Exlude, 'Subject'].as_matrix()

## Load average signal.
mean_signal = np.load('fmri_second_levels/mean_signal.npz')

for space in spaces:
    
    print space,
    
    ## Assemble design matrices.
    subj_dir = os.path.join(mri_dir, '%s', 'arc_001', 'FINAL2.%s.%s.%s' %(sm, fd, space), 'X.dat')
    scale_factors = np.array([np.loadtxt(subj_dir %subject).max(axis=0)[:len(contrasts)] 
                             for subject in subjects]).T

    for n, contrast in enumerate(contrasts):
        
        ## Load first levels.
        out_dir = 'fmri_second_levels/FINAL2.%s.%s.%s.%s' %(sm, fd, space, contrast)
        ces = np.load(os.path.join(out_dir, 'first_levels.npz'))['ces']
        
        ## Compute PSC (Pernet 2014, Frontiers in Neuroscience).
        ms = np.where(mean_signal[space], mean_signal[space], np.inf).T
        psc = np.divide(ces * scale_factors[n] * 100., ms)
        psc = prepare_image(psc.mean(axis=1), space)
        
        ## Mask image.
        fwe = nib.load(os.path.join(out_dir, 'fwe_thresh_%s.nii.gz' %threshold)).get_data()
        psc *= np.where(fwe,1,0)
        
        ## Save.
        image = nib.Nifti1Image(psc, np.load(os.path.join(out_dir,'affine.npy')))
        nib.save(image, os.path.join(out_dir, 'psc.nii.gz'))
        
print 'Done.'

## Surface Plots

In [ ]:
import os 
import numpy as np
from surfer import Brain
img_dir = 'plots/FINAL2/second_levels'
%matplotlib qt4

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
fd = 0.9
contrasts = ['Delib']
overlay = 'psc'
surface = 'inflated'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for hemi in ['lh','rh']: 

    for contrast in contrasts:

        fn = os.path.join('fmri_second_levels', 'FINAL.%s.%s.%s.%s' %(sm, fd, hemi, contrast), '%s.nii.gz' %overlay)
        
        for view in ['lateral','medial']:
            
            brain = Brain("fsaverage", hemi, surface)
            try: brain.add_overlay(fn, min=0.04, max=2.5, sign="pos")
            except: continue
            brain.show_view(view=view)
            od = os.path.join(img_dir, overlay, surface)
            if not os.path.isdir(od): os.makedirs(od)
            of = os.path.join(od, '%s_%s_%s.png' %(hemi,contrast,view))
            Brain.save_image(brain,of)

## Compute surface summary table

In [58]:
import os
import numpy as np
import nibabel as nib
from mne import Label, read_label, grow_labels, vertex_to_mni, set_log_level
set_log_level(verbose=False)
fs_dir = 'recons'
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
contrast = 'DelibMod'
thresh = 1.301
sm = 6
fd = 0.9

## ROI parameters.
extent = 10 #mm
grow = False
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

label_dir = 'fmri_second_levels/labels/seeds_%s' %contrast
labels = sorted([f for f in os.listdir(label_dir) if not f.startswith('fig') and f.endswith('label')])

fmni = open('%s/%s_surface_mni2.csv' %(label_dir,contrast), 'w')
fmni.write(','.join(['Label','V','X','Y','Z','PSC','F','p'])+'\n')

for f in labels:

    ## Extract info. Read label.
    roi, hemi = f.replace('.label', '').split('-')
    label = read_label(os.path.join(label_dir, f))
    
    ## Load accompanying overlay.
    f = 'fmri_second_levels/FINAL2.%s.%s.%s.%s/psc.nii.gz' %(sm,fd,hemi,contrast)
    overlay = nib.load(f).get_data().squeeze()

    ## Find maximum vertex.
    ix = np.argmax(overlay[label.vertices])
    v = label.vertices[ix]

    ## Extract MNI coordinates.
    x,y,z = vertex_to_mni(v, 0 if hemi=='lh' else 1, 'fsaverage', fs_dir)[0]
    
    ## Extract PSC, F-scores, p-values.
    f = 'fmri_second_levels/FINAL2.%s.%s.%s.%s/psc.nii.gz' %(sm,fd,hemi,contrast)
    psc = nib.load(f).get_data().squeeze()[v]

    f = 'fmri_second_levels/FINAL2.%s.%s.%s.%s/F.nii.gz' %(sm,fd,hemi,contrast)
    F = nib.load(f).get_data().squeeze()[v]
    
    f = 'fmri_second_levels/FINAL2.%s.%s.%s.%s/fwe_thresh_%s.nii.gz' %(sm,fd,hemi,contrast,thresh)
    p = nib.load(f).get_data().squeeze()[v]
    
    ## Write information.
    fmni.write('%s-%s,%s,%0.0f,%0.0f,%0.0f,%0.2f,%0.2f,%0.6f\n' %(roi,hemi,v,x,y,z,psc,F,10.**-p))
    
    if grow:
        
        ## Grow label.
        label = grow_labels('fsaverage', v, extent, 0 if hemi=='lh' else 1, subjects_dir=fs_dir,
                            names='fig_%s-%s' %(roi,hemi), surface='pial')[0]
        
        ## Ensure label is within actiation. Save.
        ix = np.in1d(label.vertices, np.where(overlay)[0])
        label.pos = label.pos[ix]
        label.values = label.values[ix]
        label.vertices = label.vertices[ix]
        label.save('%s/%s.label' %(label_dir, label.name))
    
fmni.close()
print 'Done.'

Done.


## Compute volume summary table

In [59]:
import os
import numpy as np
import nibabel as nib
from nibabel.affines import apply_affine
fs_dir = '/space/lilli/1/users/DARPA-Recons'
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
contrast = 'Delib'
sm = 6
fd = 0.9

## ROI parameters.
extent = 6 #mm

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
label_dir = 'fmri_second_levels/labels/seeds_%s' %contrast

## Initialize statistics file.
fmni = open('%s/%s_volume_mni2.csv' %(label_dir,contrast), 'w')
fmni.write(','.join(['Label','cX','cY','cZ','X','Y','Z','PSC','F','p'])+'\n')

## Load data. 
npz = np.load('fmri_second_levels/mni305_connectivity.npz')
affine = np.load('fmri_second_levels/FINAL2.%s.%s.mni305.%s/affine.npy' %(sm,fd,contrast))
obj = nib.load('fmri_second_levels/FINAL2.%s.%s.mni305.%s/psc.nii.gz' %(sm,fd,contrast))

overlay = obj.get_data().squeeze()
Fval = nib.load('fmri_second_levels/FINAL2.%s.%s.mni305.%s/F.nii.gz' %(sm,fd,contrast)).get_data().squeeze()
pval = nib.load('fmri_second_levels/FINAL2.%s.%s.mni305.%s/fwe_thresh_1.301.nii.gz' %(sm,fd,contrast)).get_data().squeeze()

rois = ['Left-Caudate', 'Left-Putamen', 'Left-Hippocampus',
        'Right-Caudate', 'Right-Putamen', 'Right-Hippocampus']
for roi in rois:
    
    ## Extract activated voxels in ROI.
    voxels = npz['voxels'][npz['names'] == roi]
    voxels = voxels[np.where(overlay[[arr for arr in voxels.T]])]
    
    ## Find maximally activated voxel.
    ix = np.argmax(overlay[[arr for arr in voxels.T]])
    center = voxels[ix]
    i,j,k = center
    
    ## Get MNI coordinates.
    x,y,z = apply_affine(affine, center)
    
    ## Extract max values.
    psc = overlay[i,j,k]
    F = Fval[i,j,k]
    p = pval[i,j,k]
    
    ## Write to file.
    fmni.write('%s,%0.0d,%0.0d,%0.0d,%0.0d,%0.2d,%0.2d,%0.2f,%0.2f,%0.6f\n' %(roi,i,j,k,x,y,z,psc,F,10.**-p))
    
    ## Create sphere: find all voxels within extent.
    dist = [np.linalg.norm( np.diff( apply_affine(affine,np.vstack([center,v])), axis=0 ) ) for v in voxels]
    ix = np.where(np.array(dist)<=extent)
    sphere = voxels[ix]
    
    ## Save.
    #hemi, roi = roi.split('-')
    #if hemi.startswith('L'): name = '%s-lh' %roi.lower()
    #else: name = '%s-rh' %roi.lower()
    #np.save(os.path.join(out_dir, name), sphere)
    
fmni.close()
print 'Done.'

Done.


## Post-hoc F-statistic Fix
Sam realized very late in the game he should have been saving out the pre-TFCE F-statistics. Fortunately these can be recomputed using the WLS code sans TFCE.

In [64]:
import os, sys
import numpy as np
from pandas import read_csv
from scipy.sparse import coo_matrix

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sm = 6
fd = 0.9
space = 'lh'
contrast = 'DelibMod'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

def load_sparse_coo(filename):
    npz = np.load(filename)
    M,N = npz['shape']
    return coo_matrix( (npz['data'], (npz['row'],npz['col'])), (M,N) )

out_dir = os.path.join('fmri_second_levels', 'FINAL.%s.%s.%s.%s' %(sm,fd,space,contrast))

## Load data.
npz = np.load(os.path.join(out_dir, 'first_levels.npz'))
ces = npz['ces']
cesvar = np.abs( 1. / npz['cesvar'] )

## Define indices.
connectivity = load_sparse_coo(os.path.join('fmri_second_levels', '%s_connectivity.npz' %space))
index,  = np.where(~np.isinf(cesvar).sum(axis=1).astype(bool))
include = ~np.isinf(cesvar).sum(axis=1).astype(bool)
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup for permutation testing.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load subject information.
info = read_csv(os.path.join('demographics.csv'))
info = info[~info.Exlude].reset_index()
n_subj, _ = info.shape

## Build Design Matrix.
X = np.zeros((n_subj,2))
X[:,0] = 1                                        # Intercept
X[:,1] = np.where(info.Scanner == 'Trio', 0, 1)   # Scanner
n_subj, n_pred = X.shape

sign_flips = np.ones((1,n_subj))
n_shuffles = sign_flips.shape[0]

## Preallocate arrays for results.
shape = [n_shuffles] + list(ces.shape[:-1])
Bmap = np.zeros(shape)
Fmap = np.zeros(shape)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

'''
Following the instructions of Winkler et al. (2014), we use the Freedman and Lane (1983)
permutation procedure. This allows us to precompute a number of values ahead of time.

To understand the WLS computations, please see:
https://github.com/statsmodels/statsmodels/blob/master/statsmodels/base/model.py
https://github.com/statsmodels/statsmodels/blob/master/statsmodels/regression/linear_model.py
'''

def wls(X,Y,W):
    B = np.linalg.inv(X.T.dot(W).dot(X)).dot(X.T).dot(W).dot(Y)
    ssr = W.dot( np.power(Y - np.dot(X,B),2) ).sum()
    scale = ssr / (n_subj - n_pred)
    cov_p = np.linalg.inv(X.T.dot(W).dot(X)) * scale
    F = np.power(B[0],2) * np.power(cov_p[0,0],-1)
    return B[0], F

## Loop it!
for n, sf in enumerate(sign_flips):
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    for m in index:

        ## Update variables.
        W = np.diag(cesvar[m])
        Y = ces[m]
        
        ## Permute values.
        ## See Winkler et al. (2014), pg. 385
        ## To compute Hat Matrix, see: https://en.wikipedia.org/wiki/Projection_matrix and 
        Z = X[:,1:]
        ZZ = Z.dot( np.linalg.inv( Z.T.dot(W).dot(Z) ) ).dot(Z.T).dot(W)
        Rz = np.identity(n_subj) - ZZ
        Y = np.diag(sf).dot(Rz).dot(Y)
        
        ## Perform WLS.
        Bmap[n,m], Fmap[n,m] = wls(X,Y,W) 

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save results.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#  

def prepare_image(arr, space):
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    image = np.zeros_like(npz['mapping'], dtype=float)
    
    if not space == 'mni305': 
        image[npz['vertices']] += arr
    else:
        x,y,z = npz['voxels'].T
        image[x,y,z] += arr
    
    for _ in range(4 - len(image.shape)): image = np.expand_dims(image,-1)
    return image

## Translate array back into proper space.
image = prepare_image(Fmap.squeeze(), space).squeeze()

## Load in results table.
if space == 'mni305':
    results = read_csv('fmri_second_levels/labels/seeds_%s/%s_volume_mni2.csv' %(contrast,contrast))
    fscores = [image[i,j,k] for i,j,k in results[['cX','cY','cZ']].as_matrix()]
    results['Fpre'] = fscores
    results.to_csv('fmri_second_levels/labels/seeds_%s/%s_volume_mni2.csv' %(contrast,contrast))
else:
    results = read_csv('fmri_second_levels/labels/seeds_%s/%s_surface_mni2.csv' %(contrast,contrast))
    if not 'Fpre' in results.columns: results['Fpre'] = np.nan
    vertices = results.loc[[True if label.endswith(space) else False for label in results.Label],'V'].as_matrix()
    for v in vertices: results.loc[results.V==v,'Fpre'] = image[v]
    results.to_csv('fmri_second_levels/labels/seeds_%s/%s_surface_mni2.csv' %(contrast,contrast), index=False)
    
print 'Done.'

/home/szoro/Documents/software/anaconda2.7/lib/python2.7/site-packages/ipykernel/__main__.py:30: RuntimeWarning: divide by zero encountered in divide


Done.
